## 1. 데이터 확인

설치안되어있는경우 설치 필요

In [1]:
# %pip install category_encoders

라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore') # 경고 메세지 무시

데이터 불러오기

In [3]:
df_train = pd.read_csv("./data/train.csv") # 학습용 데이터
df_test = pd.read_csv("./data/submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

## 2. 데이터 전처리

### bant_submit, business_unit, enterprise 변수

따로 처리한 부분 없음

### customer_country, customer_country.1 변수

In [5]:
def process_customer_country_tokenized(df, column_name):
    for i, entry in enumerate(df[column_name]):
        if isinstance(entry, str):
            tokens = [token.strip() for token in entry.replace('/', ',').split(',') if token.strip() != '']
            if tokens:
                df.at[i, column_name] = tokens[-1]
            else:
                df.at[i, column_name] = np.nan
        else:
            df.at[i, column_name] = np.nan
    return df

customer_country, customer_country.1 값을 토큰화 하여 말단 단어 추출

In [6]:
df_train = process_customer_country_tokenized(df_train, 'customer_country')
df_train = process_customer_country_tokenized(df_train, 'customer_country.1')

df_test = process_customer_country_tokenized(df_test, 'customer_country')
df_test = process_customer_country_tokenized(df_test, 'customer_country.1')

In [7]:
# 미국 주 이름 카테고리화
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

df_train['customer_country'] = df_train['customer_country'].replace(us_states, 'United States')
df_train['customer_country.1'] = df_train['customer_country.1'].replace(us_states, 'United States')

df_test['customer_country'] = df_test['customer_country'].replace(us_states, 'United States')
df_test['customer_country.1'] = df_test['customer_country.1'].replace(us_states, 'United States')

LG 사이트에서 작성된 나라정보를 리스트화

이때 Test데이터셋 값중 Corporate은 특정 국가나 지역을 나타내는 것이 아니라,  
보통 기업이나 조직을 나타내는 용어로 국가 리스트에 따로 포함시키지 않음

In [8]:
asia = ['Australia', 'Bangladesh', 'Brunei', 'Cambodia', 'China', 'Fiji', 'Hong Kong', 'India', 'Indonesia', 'Laos', 'Malaysia', 'Maldives', 'Myanmar', 'Nepal', 'New Zealand', 'Papula New Guinea', 'Philippines', 'Singapore', 'Sri Lanka', 'Taiwan', 'Thailand', 'Vietnam', 'Japan', 'South Korea']
cis = ['Belarus', 'Kazakhstan', 'Mongolia', 'Russia', 'Turkmenistan', 'Ukraine', 'Uzbekistan']
europe = ['Albania', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo', 'Latvia', 'Lithuania', 'Luxembourg', 'Macedonia', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'Isle of Man', 'Malta']
latin_america_and_the_caribbean = ['Anguilla', 'Antigua', 'Argentina', 'Aruba', 'Bahamas', 'Barbados', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'British Virgin Islands', 'Cayman Islands', 'Chile', 'Colombia', 'COLOMBIA', 'Costa Rica', 'Cuba', 'Curacao', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Grenada', 'Guatemala', 'Guyana Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Netherlands Antilles', 'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Puerto Rico', 'Saint Lucia', 'St Kitts', 'St Maarten', 'St Vincent', 'Suriname', 'Trinidad and Tobago', 'Turks and Caicos Islands', 'Uruguay', 'US Virgin Islands', 'Venezuela', 'Antigua and Barbuda', 'Saint Kitts and Nevis']
middle_east_and_africa = ['Afghanistan', 'Algeria', 'Angola', 'Armenia', 'Azerbaijan', 'Bahrain', 'Benin', 'Botswana', 'Burkina Faso Cameroon', 'Central African Republic', 'Congo', "Cote d'Ivoire", 'Democratic Republic of the Congo', 'Djibouti', 'Egypt', 'EGYPT', 'Equatorial Guinea', 'Ethiopia', 'Gabon', 'Gambia', 'Georgia', 'Ghana', 'Guinea Iran', 'Iraq', 'Israel', 'Ivory Coast', 'Jordan', 'Kenya', 'Kuwait', 'Lebanon', 'Liberia', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Nigeria', 'Oman', 'Pakistan', 'Palestine', 'Qatar', 'Rwanda', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Sierra Leone', 'Somalia', 'South Africa', 'Sudan', 'Swaziland', 'Syria', 'Togo', 'Tunisia', 'Türkiye', 'Turkey', 'U.A.E', 'Uganda', 'United Republic of Tanzania', 'Yemen', 'Zambia', 'Eritrea', 'Libya Malawi', 'Zimbabwe']
north_america = ['Canada', 'United States', 'UNITED STATES']

In [9]:
def update_country(country):
    if country in asia:
        return country
    elif country in cis:
        return country
    elif country in europe:
        return country
    elif country in latin_america_and_the_caribbean:
        return country
    elif country in middle_east_and_africa:
        return country
    elif country in north_america:
        return country
    else:
        return 'Unknown'

df_train['customer_country'] = df_train['customer_country'].apply(update_country)
df_test['customer_country'] = df_test['customer_country'].apply(update_country)

In [10]:
continent_dict = {country: 'Asia' for country in asia}
continent_dict.update({country: 'CIS' for country in cis})
continent_dict.update({country: 'Europe' for country in europe})
continent_dict.update({country: 'Latin America and the Caribbean' for country in latin_america_and_the_caribbean})
continent_dict.update({country: 'Middle East & Africa' for country in middle_east_and_africa})
continent_dict.update({country: 'North America' for country in north_america})

df_train['customer_country.1'] = df_train['customer_country.1'].replace(continent_dict)
df_test['customer_country.1'] = df_test['customer_country.1'].replace(continent_dict)

customer_country.1 빈도수이하값 및 결측값에 대한 처리

In [11]:
# 'customer_country.1'의 빈도 계산
counts_train = df_train['customer_country.1'].value_counts()
counts_test = df_test['customer_country.1'].value_counts()

# 일정 빈도 이하 unknown 처리
find_count_train = counts_train[counts_train <= 30].index
find_count_test = counts_test[counts_test <= 2].index

# 일정 빈도 이하인 국가를 'Unknown'으로 설정
df_train.loc[df_train['customer_country.1'].isin(find_count_train), 'customer_country.1'] = 'Other'
df_test.loc[df_test['customer_country.1'].isin(find_count_test), 'customer_country.1'] = 'Other'

# 결측값에 대한 처리
df_train['customer_country.1'] = df_train['customer_country.1'].fillna('Unknown')
df_test['customer_country.1'] = df_test['customer_country.1'].fillna('Unknown')

In [12]:
# train 데이터셋에 없는 test 데이터셋에 대한 처리
## {'Corporate'} 값이 test 데이터셋에는 존재하는 반면 train 데이터셋에는 없음
def replace_unknown_values(row, unique):
    if row not in unique:
        return 'Other'
    else:
        return row

unique_train_values = df_train['customer_country.1'].unique()
df_test['customer_country.1'] = df_test['customer_country.1'].apply(lambda x: replace_unknown_values(x, unique_train_values))

### customer_idx 변수

특정값에 대한 처리

In [13]:
# 'customer_idx'의 빈도 계산
counts_train = df_train['customer_idx'].value_counts()

# 일정 빈도 이하 처리
find_count_train = counts_train[counts_train <= 10].index

# 일정 빈도 이하인 customer_idx를 '-1'으로 설정
df_train.loc[df_train['customer_idx'].isin(find_count_train), 'customer_idx'] = -1

In [14]:
# train 데이터셋에 없는 test 데이터셋에 대한 처리
def replace_unknown_values(row, unique):
    if row not in unique:
        return -1
    else:
        return row

unique_train_values = df_train['customer_idx'].unique()
df_test['customer_idx'] = df_test['customer_idx'].apply(lambda x: replace_unknown_values(x, unique_train_values))

customer_idx의 값에 대해서 is_converted에 대한 비율 계산

In [15]:
# 'customer_idx' 카테고리별로 'is_converted'의 평균과 갯수를 계산
idx_target = df_train.groupby('customer_idx')['is_converted'].agg(['mean', 'count']).sort_values(by='mean', ascending=False)

계산한 비율에 대해서 customer_idx 값을 특정범위에 대한 범주화

In [16]:
# 'mean' 값이 0.9보다 크면 1, 아니면 0을 부여
idx_target['label'] = ((idx_target['mean'] > 0.99) & (idx_target['count'] >= 10)).astype(int)

# 'idx_target'를 기준으로 'label' 값을 매핑
idx_map = idx_target['label'].to_dict()

# 'customer_idx' 열을 업데이트
df_train['customer_idx_99'] = df_train['customer_idx'].map(idx_map)
df_test['customer_idx_99'] = df_test['customer_idx'].map(idx_map)

In [17]:
# 'mean' 값이 0.01보다 작으면 1, 아니면 0을 부여
idx_target['label'] = ((idx_target['mean'] < 0.01)& (idx_target['count'] >= 10)).astype(int)

# 'idx_target'를 기준으로 'label' 값을 매핑
idx_map = idx_target['label'].to_dict()

# 'customer_idx' 열을 업데이트
df_train['customer_idx_001'] = df_train['customer_idx'].map(idx_map)
df_test['customer_idx_001'] = df_test['customer_idx'].map(idx_map)

### customer_type 변수

오타나 연관 단어에 대해서 관련 값으로 변환

In [18]:
# 카테고리 재할당
df_train['customer_type'] = df_train['customer_type'].replace({
    'Others': 'Other',
    'Etc.': 'Other',
    'Software / Solution Provider': 'Software/Solution Provider',
    'Specifier/ Influencer': 'Specifier/Influencer',
    'Specifier / Influencer': 'Specifier/Influencer',
    'Distributor': 'Dealer/Distributor',
    'Homeowner': 'Home Owner',
    'Manager / Director' : 'Manager/Director',
    'Commercial end-user': 'End-user',
    'End-Customer': 'End Customer',
    'Consultant': 'Architect/Consultant',
    'Installer': 'Specifier/Influencer',
    'Installer/Contractor': 'Specifier/Influencer',
    
    # LG 카테고리 참고함
    'Corporate' : 'End Customer',
    'Dealer/Distributor' : 'Channel Partner',
    'Reseller' : 'Channel Partner',
    'Technician': 'Specifier/Influencer',
    'Architect/Consultant': 'Specifier/Influencer',
    'Developer': 'End Customer',  
})

df_test['customer_type'] = df_test['customer_type'].replace({
    'Specifier/ Influencer': 'Specifier/Influencer',
    'End-Customer': 'End Customer',

    # LG 카테고리 참고함
    'Developer': 'End Customer',
})

결측값 대체

In [19]:
# 'customer_type' 변수에서 NaN 값을 'Unknown' 으로 대체
df_train['customer_type'] = df_train['customer_type'].fillna('Unknown')
df_test['customer_type'] = df_test['customer_type'].fillna('Unknown')

이때 customer_type 값을 처리하는 과정에서  
End Customer가 End-user에 해당한다는것을 알수 있음으로 값을 바꿔줌

In [20]:
# 카테고리 재할당
df_train['customer_type'] = df_train['customer_type'].replace({
    'End Customer': 'End-user'
})
df_test['customer_type'] = df_test['customer_type'].replace({
    'End Customer': 'End-user'
})

### historical_existing_cnt 변수

결측값 대체

In [21]:
# median 값으로 대체(이때 median값은 4)
cnt_train_med = df_train['historical_existing_cnt'].median()
cnt_test_med = df_test['historical_existing_cnt'].median()

# median 값으로 결측값 대체
df_train['historical_existing_cnt'] = df_train['historical_existing_cnt'].fillna(cnt_train_med)
df_test['historical_existing_cnt'] = df_test['historical_existing_cnt'].fillna(cnt_test_med)

Min-Max 스케일링

In [22]:
# Min-Max 스케일링 
min_value = df_train['historical_existing_cnt'].min()
max_value = df_train['historical_existing_cnt'].max()


df_train['historical_existing_cnt'] = (df_train['historical_existing_cnt'] - min_value) / (max_value - min_value)
df_test['historical_existing_cnt'] = (df_test['historical_existing_cnt'] - min_value) / (max_value - min_value)


### id_strategic_ver, it_strategic_ver, idit_strategic_ver 변수

결측값 대체

In [23]:
# 누락된 값을 0으로 채우기
df_train['id_strategic_ver'].fillna(0, inplace=True)
df_train['it_strategic_ver'].fillna(0, inplace=True)
df_train['idit_strategic_ver'].fillna(0, inplace=True)

df_test['id_strategic_ver'].fillna(0, inplace=True)
df_test['it_strategic_ver'].fillna(0, inplace=True)
df_test['idit_strategic_ver'].fillna(0, inplace=True)

### customer_job 변수

오타관련 값에 대한 변환  
LG 홈페이지에서 제공된 job 리스트에 해당 안되는 값에 대한 변환

In [24]:
def search_job(customer_job):
    # LG list 참고
    job = ['accounting','administrative','arts and design','business development','community and social services','consulting','curation','education','engineering', 'entrepreneurship','finance','healthcare services','human resources','information technology','legal','marketing','media and communication','military and protective services operations','product management', 'program and project management','purchasing','quality assurance','real estate','research','sales','support','others']

    # (오타변환)'media and communications'를 'media and communication'으로 변환
    if customer_job == 'media and communications':
        customer_job = 'media and communication'
    
    if not customer_job:
        return 'others'
    elif customer_job in job:
        return customer_job
    else:
        return 'unknown'

df_train['customer_job'] = df_train['customer_job'].apply(search_job)
df_test['customer_job'] = df_test['customer_job'].apply(search_job)

### lead_desc_length 변수

Min-Max 스케일링

In [25]:
# Min-Max 스케일링 
min_value = df_train['lead_desc_length'].min()
max_value = df_train['lead_desc_length'].max()

df_train['lead_desc_length'] = (df_train['lead_desc_length'] - min_value) / (max_value - min_value)
df_test['lead_desc_length'] = (df_test['lead_desc_length'] - min_value) / (max_value - min_value)

결측값 대체

In [26]:
# 누락된 값을 0으로 채우기
df_train['lead_desc_length'].fillna(0, inplace=True)
df_test['lead_desc_length'].fillna(0, inplace=True)

### inquiry_type 변수

오타나 연관 단어에 대해서 관련 값으로 변환

In [27]:
# 'inquiry_type' 열의 철자오류에 대해 일관된 값으로 변환
df_train['inquiry_type'] = df_train['inquiry_type'].replace({

    'Quotation or purchase consultation': 'Quotation or Purchase Consultation',
    'Request for quotation or purchase': 'Quotation or Purchase Consultation',
    'quotation_or_purchase_consultation': 'Quotation or Purchase Consultation',
    'quotation_': 'Quotation or Purchase Consultation',
    'Purchase or Quotation': 'Quotation or Purchase Consultation',
    'Quotation or Purchase consultation': 'Quotation or Purchase Consultation',
    'Purchase': 'Quotation or Purchase Consultation',
    'quotation_or_purchase_consultation': 'Quotation or Purchase Consultation',
    'quotation_': 'Quotation or Purchase Consultation',
    'quotation_': 'Quotation or Purchase Consultation',

    'Usage or technical consultation': 'Usage or Technical Consultation',
    'Technical Consultation': 'Usage or Technical Consultation',
    'usage or technical consultation': 'Usage or Technical Consultation',
    'usage_or_technical_consultation': 'Usage or Technical Consultation',
    'Request for technical consulting': 'Usage or Technical Consultation',
    'technical_consultation': 'Usage or Technical Consultation',
    'technical': 'Usage or Technical Consultation',

    'sales':'Sales Inquiry',
    'Sales inquiry':'Sales Inquiry',

    'other': 'Other',
    'other_': 'Other',
    'others': 'Other',
    'Others' : 'Other',
    'Etc.': 'Other',
    'ETC.': 'Other'

})

df_test['inquiry_type'] = df_test['inquiry_type'].replace({

    'Technical Consultation': 'Usage or Technical Consultation',

    'other_': 'Other',
    'Others' : 'Other',

})

특정값에 대한 처리('Other' 값으로 변환)

In [28]:
# 'inquiry_type' 열의 값별 개수를 계산합니다.
value_counts = df_train['inquiry_type'].value_counts()

# 2개 이하인 값들의 리스트를 만듭니다.
to_replace = value_counts[value_counts <= 2].index

# 2개 이하인 값들을 'Other'로 업데이트합니다.
df_train['inquiry_type'] = df_train['inquiry_type'].replace(to_replace, 'Other')

In [29]:
# train 데이터셋에 없는 test 데이터셋에 대한 처리
## test 데이터셋 값중 {'Media Inquiry'} 값에 대한 처리
def replace_unknown_values(row, unique):
    if row not in unique:
        return 'Other'
    else:
        return row

unique_train_values = df_train['inquiry_type'].unique()
df_test['inquiry_type'] = df_test['inquiry_type'].apply(lambda x: replace_unknown_values(x, unique_train_values))

결측값 대체('Unknown' 값으로 변환)

In [30]:
# 'inquiry_type' 변수에서 NaN 값을 'Unknown' 으로 대체
df_train['inquiry_type'] = df_train['inquiry_type'].fillna('Unknown')
df_test['inquiry_type'] = df_test['inquiry_type'].fillna('Unknown')

### product_category 변수

특정값에 대해서 관련 값으로 변환

In [31]:
df_train['product_category'] = df_train['product_category'].replace({
    'notebook': 'laptop',
    'others': 'other',
    'ess': 'other',
    'signage care solution': 'other',
})

df_test['product_category'] = df_test['product_category'].replace({
    'notebook': 'laptop',
    'others': 'other',
    'ess': 'other',
    'signage care solution': 'other',
})

특정 빈도수 이하 값대체

In [32]:
# 'product_category'의 빈도 계산
counts_train = df_train['product_category'].value_counts()

# 빈도가 특정수치 이하인 product_category 를 찾음
find_count_train = counts_train[counts_train <= 15].index

# 빈도가 특정수치 이하값 
df_train.loc[df_train['product_category'].isin(find_count_train), 'product_category'] = 'other'

결측값 대체

In [33]:
# 'product_category' 변수에서 NaN 값을 'Unknown' 으로 대체
df_train['product_category'] = df_train['product_category'].fillna('unknown')
df_test['product_category'] = df_test['product_category'].fillna('unknown')

### product_subcategory, product_modelname 변수

train 데이터셋에 없는 데이터에 대한 처리

In [34]:
# train 데이터셋에 없는 test 데이터셋에 대한 처리
def replace_unknown(row, unique):
    if row not in unique:
        return 'Unknown'
    else:
        return row

# product_subcategory 결측값 대체
unique_subcategory = df_train['product_subcategory'].unique()
df_test['product_subcategory'] = df_test['product_subcategory'].apply(lambda x: replace_unknown(x, unique_subcategory))

# product_modelname 결측값 대체
unique_modelname = df_train['product_modelname'].unique()
df_test['product_modelname'] = df_test['product_modelname'].apply(lambda x: replace_unknown(x, unique_modelname))

결측값 대체

In [35]:
# 'product_subcategory' 변수에서 NaN 값을 'Unknown' 으로 대체
df_train['product_subcategory'] = df_train['product_subcategory'].fillna('Unknown')
df_test['product_subcategory'] = df_test['product_subcategory'].fillna('Unknown')

# 'product_modelname' 변수에서 NaN 값을 'Unknown' 으로 대체
df_train['product_modelname'] = df_train['product_modelname'].fillna('Unknown')
df_test['product_modelname'] = df_test['product_modelname'].fillna('Unknown')

### customer_position 변수

오타에 대한 값이나 관련값으로 대체가능한 값에 대한 처리

In [36]:
df_train['customer_position'] = df_train['customer_position'].replace({
    'vicepresident': 'vice president',
    'vp': 'vice president',
    'entrylevel': 'entry level',
    'c-levelexecutive': 'c-level executive',
    'founder': 'ceo/founder',
    'ceo/fundador': 'ceo/founder',
    'commercial consultant': 'consultant',
    'architect/consultant': 'consultant',
    'architecture/consult': 'consultant',
    'business unit director': 'director',
    'no influence': 'none',
    'not applicable': 'none',
    'commercial end-user': 'end-user',
    'exhibitiontv': 'exhibition',
    'decision-influencer': 'decision influencer',
    'sales': 'business development/sales',
    'subsidiary sales (ise)': 'business development/sales',
    'business development': 'business development/sales',
    'medical device manufacturer': 'manufacturer',
    'assistant professor of enlish': 'assistant professor',
    'asst prof.': 'assistant professor',
    'prof.': 'professor',
    'professor of mathematics': 'professor',
    'principal & director': 'director',
    'others': 'other',
    'decision-maker': 'decision maker',
})

df_test['customer_position'] = df_test['customer_position'].replace({
    'vicepresident': 'vice president',
    'vp': 'vice president',
    'entrylevel': 'entry level',
    'c-levelexecutive': 'c-level executive',
    'founder': 'ceo/founder',
    'ceo/fundador': 'ceo/founder',
    'commercial consultant': 'consultant',
    'architect/consultant': 'consultant',
    'architecture/consult': 'consultant',
    'business unit director': 'director',
    'no influence': 'none',
    'not applicable': 'none',
    'commercial end-user': 'end-user',
    'exhibitiontv': 'exhibition',
    'decision-influencer': 'decision influencer',
    'sales': 'business development/sales',
    'subsidiary sales (ise)': 'business development/sales',
    'business development': 'business development/sales',
    'medical device manufacturer': 'manufacturer',
    'assistant professor of enlish': 'assistant professor',
    'asst prof.': 'assistant professor',
    'prof.': 'professor',
    'professor of mathematics': 'professor',
    'principal & director': 'director',
    'others': 'other'
})

특정 범주이하의 값에 대한 처리

In [37]:
## train
# 'inquiry_type' 열의 값별 개수 계산
counts_train = df_train['customer_position'].value_counts()
counts_test = df_test['customer_position'].value_counts()

# 특정수 이하인 값들의 리스트로 만듬
replace_train = counts_train[counts_train <= 40].index
replace_test = counts_test[counts_test <= 2].index

# 특정수 이하인 값들을 'other'로 업데이트
df_train['customer_position'] = df_train['customer_position'].replace(replace_train, 'other')
df_test['customer_position'] = df_test['customer_position'].replace(replace_test, 'other')

### response_corporate 변수

특정 범주이하 값 처리

In [38]:
# 'response_corporate'의 빈도 계산
counts_train = df_train['response_corporate'].value_counts()

# 빈도가 특정수치 이하인 product_category 를 찾음
find_count_train = counts_train[counts_train <= 5].index

# 빈도가 특정수치 이하값 
df_train.loc[df_train['response_corporate'].isin(find_count_train), 'response_corporate'] = 'other'

train 데이터셋에 없는 값에 대한 처리

In [39]:
# train 데이터셋에 없는 test 데이터셋에 대한 처리
def replace_unknown_values(row, unique):
    if row not in unique:
        return 'other'
    else:
        return row

unique_train_values = df_train['response_corporate'].unique()
df_test['response_corporate'] = df_test['response_corporate'].apply(lambda x: replace_unknown_values(x, unique_train_values))

### expected_timeline 변수

'months' , 'year'를 포함하는 값을 추출  
(이때 해당안되는 값에 대해서는 etc 값으로 변환)

In [40]:
# 'months' , 'year' 포함하는 값 추출(아닌경우 'etc' 값으로 변환)
df_train['expected_timeline'] = np.where(df_train['expected_timeline'].str.contains('months|year', na=False), df_train['expected_timeline'], 'etc.')
df_test['expected_timeline'] = np.where(df_test['expected_timeline'].str.contains('months|year', na=False), df_test['expected_timeline'], 'etc.')

추출한 값에 대해서 관련 값으로 변환

In [41]:
df_train['expected_timeline'] = df_train['expected_timeline'].replace({
    'less_than_3_months': 'less than 3 months',
    '3_months_~_6_months': '3 months ~ 6 months',
    '9_months_~_1_year': '9 months ~ 1 year',
    'more_than_a_year': 'more than a year',
    '6_months_~_9_months': '6 months ~ 9 months',
    '9 months - 1 year': '9 months ~ 1 year',
    'duplicate lead - il220100042906. less than 3 months': 'less than 3 months',
    'less than 3 months ,meeting with the customer for the more details and tentative boq will ne 32 and 43': 'less than 3 months',

    'less_than_3_months': 'less than 3 months',
    'less than 3 months- outdoor led requiment': 'less than 3 months',
    'need to discuss with client in next two months. they need to check the product and accridngly proceed for personal use.': 'less than 3 months',
    'quotation sent – 75tr3dj , work in progress, he will buy after 2 months. he has not even seen the quote yet.': 'less than 3 months',
    'quote shared with customer, he will confirm after 2 months. lead shared with partner.': 'less than 3 months',
    'less than 3 months- outdoor led requiment': 'less than 3 months',
    'less than 3 months. customer not answered . to call back': 'less than 3 months',

    'purchase planning after 3 months': '3 months ~ 6 months',
    'needs hotel tv after 4 months, will call us.': '3 months ~ 6 months',

    'we are already in touch with this cutsomer from last 2 years, he has never purchased any product till date. i called him up twice but no reponse.' : 'etc.',
    'very abrupt customer. said the inquiry was made months ago and was rude enough. closing in the system as the client behaviour has no scope to discuss on requirement. need marketing team to check if the case was received in dec or jan as per client.' : 'etc.',

    # 아래 값들은 근사 기간으로 넣어줌
    'more then 3 months': '3 months ~ 6 months',
    'less than 5 months': '3 months ~ 6 months',
    'less than 6 months': '3 months ~ 6 months',
    'less then 6 months': '3 months ~ 6 months',
    '3 months': '3 months ~ 6 months',
    '4/8 months': '3 months ~ 6 months',

})
df_test['customer_position'] = df_test['customer_position'].replace({
    'less_than_3_months': 'less than 3 months',
    '3_months_~_6_months': '3 months ~ 6 months',
    '9_months_~_1_year': '9 months ~ 1 year',
    'more_than_a_year': 'more than a year',
    '6_months_~_9_months': '6 months ~ 9 months'
})


### ver_cus, ver_pro 변수

따로처리할 부분 없음

### ver_win_rate_x 변수

결측값 대체

In [42]:
# 'ver_win_rate_x' 변수에서 NaN 값을 0으로 대체
df_train['ver_win_rate_x'] = df_train['ver_win_rate_x'].fillna(0)
df_test['ver_win_rate_x'] = df_test['ver_win_rate_x'].fillna(0)

### ver_win_ratio_per_bu 변수

business_area, business_unit 의 변수로 그룹화 진행  
해당하는 그룹에서 'ver_win_ratio_per_bu' 의 결측값에 대한 예측

In [43]:
grouped = df_train.groupby(["business_area","business_unit"])

# business_area, business_unit 그룹화하여 샘플 수 계산
grouped_counts = df_train.groupby(['business_area', 'business_unit']).size()

# business_area, business_unit, is_converted 그룹화하여 영업 전환된(is_converted) 샘플 수 계산
converted_counts = df_train[df_train['is_converted']].groupby(['business_area', 'business_unit']).size()

# business_area, business_unit별 샘플 수 대비 영업 전환된(is_converted) 샘플 수의 비율 계산
conversion_rates = converted_counts / grouped_counts

위의 코드를 통해 구한 값에 대해서 해당하는 부분의 값이 결측값인 경우 값대체

In [44]:
# 그룹별 값을 딕셔너리로 정의
group_values = {
    ('corporate / office', 'AS'): 0.029734,
    ('corporate / office', 'ID'): 0.088618,
    ('corporate / office', 'IT'): 0.033333,
    ('corporate / office', 'Solution'): 0.034483,
    ('education', 'AS'): 0.066667,
    ('education', 'ID'): 0.064897,
    ('education', 'IT'): 0.046667,
    ('education', 'Solution'): None,
    ('factory', 'AS'): 0.059553,
    ('factory', 'ID'): 0.068519,
    ('factory', 'IT'): 0.337349,
    ('factory', 'Solution'): None,
    ('government department', 'AS'): 0.040462,
    ('government department', 'ID'): 0.076010,
    ('government department', 'IT'): None,
    ('government department', 'Solution'): None,
    ('hospital & health care', 'AS'): 0.096154,
    ('hospital & health care', 'ID'): 0.128378,
    ('hospital & health care', 'IT'): 0.378771,
    ('hotel & accommodation', 'AS'): 0.004831,
    ('hotel & accommodation', 'ID'): 0.118902,
    ('hotel & accommodation', 'IT'): 0.002528,
    ('hotel & accommodation', 'Solution'): None,
    ('power plant / renewable energy', 'AS'): 0.129032,
    ('power plant / renewable energy', 'ID'): 0.279070,
    ('power plant / renewable energy', 'IT'): None,
    ('public facility', 'AS'): 0.030000,
    ('public facility', 'ID'): 0.099631,
    ('public facility', 'IT'): 0.025000,
    ('public facility', 'Solution'): None,
    ('residential (home)', 'AS'): 0.017582,
    ('residential (home)', 'ID'): 0.038961,
    ('residential (home)', 'IT'): 0.142857,
    ('residential (home)', 'Solution'): None,
    ('retail', 'AS'): 0.026650,
    ('retail', 'ID'): 0.061637,
    ('retail', 'IT'): 0.073620,
    ('retail', 'Solution'): None,
    ('special purpose', 'AS'): 0.028050,
    ('special purpose', 'ID'): 0.070698,
    ('special purpose', 'IT'): 0.046296,
    ('special purpose', 'Solution'): None,
    ('transportation', 'AS'): 0.037736,
    ('transportation', 'ID'): 0.064815,
    ('transportation', 'IT'): 0.060606,
    ('transportation', 'Solution'): None,
}

# 'business_area'와 'business_unit'에 해당하는 그룹별 값을 'ver_win_ratio_per_bu' 변수에 적용
df_train['ver_win_ratio_per_bu'] = df_train.apply(lambda row: group_values.get((row['business_area'], row['business_unit'])), axis=1)
df_test['ver_win_ratio_per_bu'] = df_test.apply(lambda row: group_values.get((row['business_area'], row['business_unit'])), axis=1)

결측값 대체

In [45]:
# 결측값 0값으로 대체
df_train['ver_win_ratio_per_bu'].fillna(0, inplace=True)
df_test['ver_win_ratio_per_bu'].fillna(0, inplace=True)

### business_area, business_subarea 변수

결측값 대체

In [46]:
# 결측값 'unknown' 값으로 대체
df_train['business_area'].fillna('unknown', inplace=True)
df_test['business_area'].fillna('unknown', inplace=True)

# 결측값 'Unknown' 값으로 대체
df_train['business_subarea'].fillna('Unknown', inplace=True)
df_test['business_subarea'].fillna('Unknown', inplace=True)

### lead_owner 변수

특정 빈도수 이하 값에 대한 처리

In [47]:
# 'lead_owner'의 빈도 계산
counts_train = df_train['lead_owner'].value_counts()

# 빈도가 특정수치 이하인 lead_owner 를 찾음
find_count_train = counts_train[counts_train <= 10].index

# 빈도가 특정수치 이하값 
df_train.loc[df_train['lead_owner'].isin(find_count_train), 'lead_owner'] = -1

결측값 대체

In [48]:
# train 데이터셋에 없는 test 데이터셋에 대한 처리
def replace_unknown_values(row, unique):
    if row not in unique:
        return -1
    else:
        return row

unique_train_values = df_train['lead_owner'].unique()
df_test['lead_owner'] = df_test['lead_owner'].apply(lambda x: replace_unknown_values(x, unique_train_values))

lead_owner에 대하여 카테고리화하여 is_converted의 비율을 계산  
계산한 값에대해서 특정 비율에 해당하는 값에대해서 범주화 진행

In [49]:
# 'customer_idx' 카테고리별로 'is_converted'의 평균과 갯수를 계산
owner_target = df_train.groupby('lead_owner')['is_converted'].agg(['mean', 'count']).sort_values(by='mean', ascending=False)

# 'mean' 값이 특정 값보다 크면 1, 아니면 0을 부여
owner_target['label_09'] = (owner_target['mean'] > 0.9).astype(int)
owner_target['label_07'] = ((0.9 >= owner_target['mean']) & (owner_target['mean'] > 0.7)).astype(int)
owner_target['label_05'] = ((0.7 >= owner_target['mean']) & (owner_target['mean'] > 0.5)).astype(int)
owner_target['label_03'] = ((0.5 >= owner_target['mean']) & (owner_target['mean'] > 0.3)).astype(int)
owner_target['label_00'] = ((0.001 >= owner_target['mean']) & (owner_target['count'] <= 100)).astype(int)


# 'owner_target'를 기준으로 'label' 값을 매핑
label_map_09 = owner_target['label_09'].to_dict()
label_map_07 = owner_target['label_07'].to_dict()
label_map_05 = owner_target['label_05'].to_dict()
label_map_03 = owner_target['label_03'].to_dict()
label_map_00 = owner_target['label_00'].to_dict()


# 'lead_owner' 열을 업데이트
df_train['lead_owner_09'] = df_train['lead_owner'].map(label_map_09)
df_train['lead_owner_07'] = df_train['lead_owner'].map(label_map_07)
df_train['lead_owner_05'] = df_train['lead_owner'].map(label_map_05)
df_train['lead_owner_03'] = df_train['lead_owner'].map(label_map_03)
df_train['lead_owner_00'] = df_train['lead_owner'].map(label_map_00)

df_test['lead_owner_09'] = df_test['lead_owner'].map(label_map_09)
df_test['lead_owner_07'] = df_test['lead_owner'].map(label_map_07)
df_test['lead_owner_05'] = df_test['lead_owner'].map(label_map_05)
df_test['lead_owner_03'] = df_test['lead_owner'].map(label_map_03)
df_test['lead_owner_00'] = df_test['lead_owner'].map(label_map_00)

## 3. 레이블 인코딩

In [50]:
# 인코딩할 컬럼 목록
columns_to_encode = ["customer_country",
                     "business_subarea",
                     "business_area",
                     "business_unit",
                     "customer_type",
                     "enterprise",
                     "customer_job",
                     "inquiry_type",
                     "product_category",
                     "product_subcategory",
                     "product_modelname",
                     "customer_country.1",
                     "customer_position",
                     "response_corporate",
                     "expected_timeline",
                     "customer_idx",
                     "lead_owner"
                     ]

df_train_encoded = df_train.copy()
df_test_encoded = df_test.copy()

for col in columns_to_encode:
    df_train_encoded[col] = df_train_encoded[col].astype('category')
    df_test_encoded[col] = df_test_encoded[col].astype('category')

타겟인코딩을 통해 범주형 변수들 수치형으로 변환

In [51]:
from category_encoders import TargetEncoder

smoothing1_list = ['enterprise','response_corporate','business_unit','business_area', 'customer_country.1']
smoothing2_list = ['lead_owner', 'customer_idx', 'customer_country']
smoothing3_list = [item for item in columns_to_encode if item not in smoothing1_list + smoothing2_list]

for col in smoothing1_list:
    encoders = {col: TargetEncoder(smoothing=1) for col in smoothing1_list}
    df_train_encoded[col] = encoders[col].fit_transform(df_train_encoded[col], df_train['is_converted'])
    df_test_encoded[col] = encoders[col].transform(df_test_encoded[col])

for col in smoothing2_list:
    encoders = {col: TargetEncoder(smoothing=5) for col in smoothing2_list}
    df_train_encoded[col] = encoders[col].fit_transform(df_train_encoded[col], df_train['is_converted'])
    df_test_encoded[col] = encoders[col].transform(df_test_encoded[col])

for col in smoothing3_list:
    encoders = {col: TargetEncoder(smoothing=10) for col in smoothing3_list}
    df_train_encoded[col] = encoders[col].fit_transform(df_train_encoded[col], df_train['is_converted'])
    df_test_encoded[col] = encoders[col].transform(df_test_encoded[col])


### com_reg_ver_win_rate 변수

범주형 데이터에 대해서 수치형으로 변환을 함  
이에 com_reg_ver_win_rate 결측값에 대한 예측이 가능해짐

랜덤포레스트의 회귀모델을 이용하여 결측값을 예측하여 결측값 대체

In [52]:
from sklearn.ensemble import RandomForestRegressor

def fill_missing_values(df):
    # 데이터에서 결측치가 없는 행과 결측치가 있는 행 분리
    train_data = df[df['com_reg_ver_win_rate'].notna()]
    test_data = df[df['com_reg_ver_win_rate'].isna()]

    # 'com_reg_ver_win_rate'를 예측하는 데 사용할 피처 선택
    features = ['business_area', 'business_unit', 'customer_country.1']   

    # 훈련 데이터와 테스트 데이터 준비
    X_train = train_data[features]
    y_train = train_data['com_reg_ver_win_rate']
    X_test = test_data[features]

    # 랜덤 포레스트 모델 생성 및 훈련
    model = RandomForestRegressor(random_state=0)
    model.fit(X_train, y_train)

    # 모델을 사용해 결측치 예측
    predicted_values = model.predict(X_test)

    # 예측값으로 결측치 대체
    df.loc[df['com_reg_ver_win_rate'].isna(), 'com_reg_ver_win_rate'] = predicted_values


# df_train_encoded에 대한 데이터 처리
fill_missing_values(df_train_encoded)

# df_test_encoded에 대한 데이터 처리
fill_missing_values(df_test_encoded)

상관관계와 다중공산성을 판단하여 결측값이 많은 product_modelname 변수와  
의미없는 변수라고 생각되는 customer_country 변수에 대해서 drop을 진행

In [53]:
df_train_encoded = df_train_encoded.drop(['customer_country','product_modelname'], axis=1)
df_test_encoded = df_test_encoded.drop(['customer_country','product_modelname'], axis=1) 

In [54]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   business_unit            59299 non-null  float64
 2   com_reg_ver_win_rate     59299 non-null  float64
 3   customer_idx             59299 non-null  float64
 4   customer_type            59299 non-null  float64
 5   enterprise               59299 non-null  float64
 6   historical_existing_cnt  59299 non-null  float64
 7   id_strategic_ver         59299 non-null  float64
 8   it_strategic_ver         59299 non-null  float64
 9   idit_strategic_ver       59299 non-null  float64
 10  customer_job             59299 non-null  float64
 11  lead_desc_length         59299 non-null  float64
 12  inquiry_type             59299 non-null  float64
 13  product_category         59299 non-null  float64
 14  product_subcategory   

## 4. 모델링

### 오버샘플링 및 언더샘플링

타겟변수에 대한 값을 변환  
(데이터의 오버샘플링과 언더샘플링을 적용하기 위함)

In [55]:
from sklearn.model_selection import train_test_split

# 타겟변수에 대한 값 변환
df_train_encoded.loc[df_train_encoded['is_converted'] == True, 'is_converted'] = 1
df_train_encoded.loc[df_train_encoded['is_converted'] == False, 'is_converted'] = 0

df_train_encoded['is_converted'] = df_train_encoded['is_converted'].astype(float)

X = df_train_encoded[df_train_encoded.columns.drop('is_converted')]
Y = df_train_encoded['is_converted']

# X와 Y로 나누기
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=0, shuffle=True)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted"], axis=1)

In [56]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# 오버샘플링과 언더샘플링
# SMOTE와 RandomUnderSampler를 파이프라인으로 결합
resample = Pipeline([('SMOTE', SMOTE(random_state=0)), 
                     ('RandomUnderSampler', RandomUnderSampler(random_state=0))])

# 데이터에 오버샘플링과 언더샘플링 적용
x_train, y_train = resample.fit_resample(x_train, y_train)

### 모델 라이브러리

In [57]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

### 모델 성능확인

In [58]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)

### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [59]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 800)
        , 'depth': trial.suggest_int('depth', 2, 16)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2)
        , 'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.01, 10.0)
        , 'border_count': trial.suggest_int('border_count', 50, 400)

        , 'random_seed': 0
        , 'verbose': False
    }
    
    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=500)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-24 12:50:17,822] A new study created in memory with name: no-name-82ee072e-91cf-4479-82d0-ea38857db68d
[I 2024-02-24 12:51:56,763] Trial 0 finished with value: 0.8308934881373045 and parameters: {'iterations': 484, 'depth': 12, 'learning_rate': 0.12452504145361233, 'l2_leaf_reg': 0.4311710058685491, 'border_count': 198}. Best is trial 0 with value: 0.8308934881373045.
[I 2024-02-24 12:52:09,018] Trial 1 finished with value: 0.8242424242424243 and parameters: {'iterations': 552, 'depth': 8, 'learning_rate': 0.17943687014859516, 'l2_leaf_reg': 7.780155576901417, 'border_count': 184}. Best is trial 0 with value: 0.8308934881373045.
[I 2024-02-24 12:52:24,078] Trial 2 finished with value: 0.8287013643254169 and parameters: {'iterations': 654, 'depth': 9, 'learning_rate': 0.11792846660784713, 'l2_leaf_reg': 5.981221901152554, 'border_count': 74}. Best is trial 0 with value: 0.8308934881373045.
[I 2024-02-24 12:52:25,648] Trial 3 finished with value: 0.7608994951812759 and paramet

: 

In [ ]:
# CatBoost
cat_model = CatBoostClassifier(
    iterations=628
    , depth=9
    , learning_rate=0.15969576473215213
    , l2_leaf_reg= 0.02016777824287572
    , border_count = 160
    , verbose=False
    , random_state=0
)

cat_model.fit(x_train, y_train)

pred = cat_model.predict(x_val)
get_clf_eval(y_val, pred)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = cat_model.predict(x_test)
sum(test_pred) # True로 예측된 개수

In [ ]:
# CatBoost
cat_model = CatBoostClassifier(
    iterations=650
    , depth=9
    , learning_rate=0.1628656915939138
    , l2_leaf_reg= 0.020222100142839153
    , border_count = 152
    , verbose=False
    , random_state=0
)

cat_model.fit(x_train, y_train)

pred = cat_model.predict(x_val)
get_clf_eval(y_val, pred)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = cat_model.predict(x_test)
sum(test_pred) # True로 예측된 개수

In [60]:
# CatBoost
cat_model = CatBoostClassifier(
    iterations=627
    , depth=8
    , learning_rate=0.16672163155553238
    , l2_leaf_reg= 0.019556805887189933
    , border_count = 163
    , verbose=False
    , random_state=0
)

cat_model.fit(x_train, y_train)

pred = cat_model.predict(x_val)
get_clf_eval(y_val, pred)

# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = cat_model.predict(x_test)
sum(test_pred) # True로 예측된 개수


오차행렬:


,True,False
True,817,177
False,126,10740


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.974452,0.866384,0.821932,0.843573,0.974146


872

.